In [1]:
import os

import pandas as pd
import time
import pickle
import copy

In [2]:

from ase.calculators.espresso import Espresso, EspressoProfile
from ase.io.espresso import read_espresso_out

In [3]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [4]:
from ase.io.espresso import read_espresso_out

In [5]:
profile = EspressoProfile(
    command='mpirun -np 4 --allow-run-as-root /usr/qe/bin/pw.x', 
    pseudo_dir='/usr/src/SSSP_efficiency/'
)

pseudopotentials = {
    'Al':'Al.pbe-n-kjpaw_psl.1.0.0.UPF',
    'Co':'Co_pbe_v1.2.uspp.F.UPF',
    'Cr':'cr_pbe_v1.5.uspp.F.UPF',
    'Cu':'Cu.paw.z_11.ld1.psl.v1.0.0-low.upf',
    #'Cu':'Cu.pbe-kjpaw.UPF',
    'Fe':'Fe.pbe-spn-kjpaw_psl.0.2.1.UPF',
    'Mn':'mn_pbe_v1.5.uspp.F.UPF',
    'Ni':'ni_pbe_v1.4.uspp.F.UPF',
    'V':'v_pbe_v1.4.uspp.F.UPF',
     }


input_data = {
    'system': {'ecutwfc': 60, 'ecutrho': 480},
    'disk_io': 'low',  # Automatically put into the 'control' section
    'tstress':True,  # deprecated, put in input_data
    'tprnfor':True,  # deprecated, put in input_data
    'calculation': 'vc-relax',#'scf',
    'conv_thr':1e-8,
    'occupations':'smearing',
    'smearing':'mv',
    'degauss':0.02,
    'mixing_beta':0.3,
    #'diagonalization':'cg',
}

calc_espresso_relax = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    input_data=input_data,
    kspacing=0.5e-1,
)

def parse_calculation_relax(file_name = 'espresso.pwo'):
    with open(file_name, 'r') as f:
        for i in read_espresso_out(f,):
            pass
    return i.copy(), i.get_potential_energy()

def parse_calculation_traj_relax(file_name = 'espresso.pwo'):
    res = []
    with open(file_name, 'r') as f:
        for i in read_espresso_out(f,):
            res.append([i.copy(), i.get_potential_energy(), i.get_forces(), i.get_stress()])
    return res

def relaxer_espresso(row):
    structure = row['init_structure']
    structure_for_relax = copy.deepcopy(structure)
    structure_for_relax.calc = calc_espresso_relax
    structure_for_relax.get_potential_energy()
    return parse_calculation_traj_relax()
    
def relaxer_espresso_jitter_ia(row, ia = 0.01):
    structure = row['init_structure']
    structure_for_relax = copy.deepcopy(structure)
    structure_for_relax = jitter_ia(structure_for_relax, ia)
    structure_for_relax.calc = calc_espresso_relax
    structure_for_relax.get_potential_energy()
    return parse_calculation_traj_relax()
    
def relaxer_espresso_jitter_ic(row, ic = 0.005):
    structure = row['init_structure']
    structure_for_relax = copy.deepcopy(structure)
    structure_for_relax = jitter_ic(structure_for_relax, ic)
    # structure.set_cell(structure.cell.array, scale_atoms = True)
    structure_for_relax.calc = calc_espresso_relax
    structure_for_relax.get_potential_energy()
    return parse_calculation_traj_relax()

def relaxer_espresso_jitter(row):
    structure = row['init_structure']
    structure_for_relax = copy.deepcopy(structure)
    structure_for_relax = jitter_ic(structure_for_relax)
    structure_for_relax = jitter_ia(structure_for_relax)
    # structure.set_cell(structure.cell.array, scale_atoms = True)
    structure_for_relax.calc = calc_espresso_relax
    structure_for_relax.get_potential_energy()
    return parse_calculation_traj_relax()

In [6]:
from disalloy.grid_run import run_packet

In [7]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [8]:
df = structure_DB.copy()
# if os.path.exists('df_espresso.pkl'):
#     df = pd.read_pickle('df_espresso.pkl')

In [9]:
relaxer_dic_espresso = {'calculator_label': 'relaxer_espresso',
                            'relaxer': relaxer_espresso}

In [35]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-06 12:53:49 - starting packet run: attempting 10 runs of size 1

2024-09-06 12:56:13 - index of ran strctures:
 [0, 1, 2, 4, 5, 7]

2024-09-06 12:56:13 - Succesfuly ran packet of 6 runs in 112.22425985336304s

144.19483137130737s


In [53]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso, 20, 2)
print('{}s'.format(time.time() - ti))

2024-09-06 15:37:56 - starting packet run: attempting 20 runs of size 2

2024-09-06 16:40:28 - index of ran strctures:
 [58, 63, 16, 42, 57, 28, 45, 62, 37, 13, 19, 26, 22, 44, 9, 17, 12, 51, 14, 34]

2024-09-06 16:40:28 - Succesfuly ran packet of 20 runs in 3727.194798231125s

3751.674234867096s


In [52]:
df[df[relaxer_dic_espresso['calculator_label']].notnull()]

,init_structure,size,relaxer_espresso,relaxer_espresso_jitter
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Al', [0.0, 0.0, 0.0], index=0)), -531...","[[(Atom('Al', [-0.00036055481894358506, -0.003..."
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Co', [0.0, 0.0, 0.0], index=0)), -405...","[[(Atom('Co', [-0.007120012382663079, -0.00308..."
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Cr', [0.0, 0.0, 0.0], index=0)), -238...","[[(Atom('Cr', [0.005919105061970768, 0.0001801..."
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Cu', [0.0, 0.0, 0.0], index=0)), -289...","[[(Atom('Cu', [-0.006625174199230526, -0.00604..."
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Fe', [0.0, 0.0, 0.0], index=0)), -447...","[[(Atom('Fe', [-0.0057916214501394165, -0.0013..."
5,"(Atom('Mn', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Mn', [0.0, 0.0, 0.0], index=0)), -287...","[[(Atom('Mn', [-0.0022240126870682136, -0.0052..."
6,"(Atom('Ni', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('Ni', [0.0, 0.0, 0.0], index=0)), -466...","[[(Atom('Ni', [0.0068131338603476125, -0.00447..."
7,"(Atom('V', [0.0, 0.0, 0.0], index=0))",1,"[[(Atom('V', [0.0, 0.0, 0.0], index=0)), -1967...","[[(Atom('V', [0.006225768253930826, -0.0046449..."
8,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"[[(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom(...",None
10,"(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom('C...",2,"[[(Atom('Al', [0.0, 0.0, 0.0], index=0), Atom(...",None


In [47]:
for i in df.loc[:7, 'init_structure']:
    print(i.cell.array)

[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]


In [48]:
calculated_index = df[df[relaxer_dic_espresso[
    'calculator_label']].notnull()].index

In [49]:
for i, j  in zip(
    df.loc[calculated_index, 'init_structure'], 
    df.loc[calculated_index, 'relaxer_espresso'].array):
    print(i.get_chemical_symbols())
    print(i.cell.array)
    print()
    print(j[-1][0].cell.array)
    
    print()
    if False:
        print('ijs')
        for ij in j:
            print('ij')
            print(ij[0].cell.array)
            print(ij[1])
            print(ij[2])
            print(ij[3])
            print('@')
            print()
    print(compute_distortion(i, j[-1][0]))
    print()
    print('#'*50)

['Al']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[2.02082085 2.02082085 0.        ]
 [0.         2.02082085 2.02082085]
 [2.02082085 0.         2.02082085]]

1.7281680248448638e-16

##################################################
['Co']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[1.72412766 1.72412766 0.        ]
 [0.         1.72412766 1.72412766]
 [1.72412766 0.         1.72412766]]

1.7281680248448638e-16

##################################################
['Cr']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[1.81085665 1.81085665 0.        ]
 [0.         1.81085665 1.81085665]
 [1.81085665 0.         1.81085665]]

2.434998949256621e-16

##################################################
['Cu']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[1.80982878 1.80982878 0.        ]
 [0.         1.80982878 1.80982878]
 [1.80982878 0.         1.80982878]]

1.7281680248448638e-16

##################################################
['Fe']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.

In [41]:
relaxer_dic_espresso_jitter = {
    'calculator_label': 'relaxer_espresso_jitter',
    'relaxer': relaxer_espresso_jitter}

In [42]:
ti = time.time()
run_packet('df_espresso', df, relaxer_dic_espresso_jitter, 10, 1,
          copy_calculated=relaxer_dic_espresso['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-06 12:57:28 - starting packet run: attempting 10 runs of size 1

2024-09-06 13:32:41 - index of ran strctures:
 [0, 1, 2, 4, 5, 6, 7]

2024-09-06 13:32:41 - Succesfuly ran packet of 7 runs in 2086.1792826652527s

2113.194128036499s


In [43]:
calculated_index = df[df[relaxer_dic_espresso_jitter[
    'calculator_label']].notnull()].index

In [33]:
for i, j, k  in zip(
    df.loc[calculated_index, 'init_structure'], 
    df.loc[calculated_index, 'relaxer_espresso'].array,
    df.loc[calculated_index, 'relaxer_espresso_jitter'].array):
    print(i.get_chemical_symbols())
    print(i.cell.array)
    print()
    print(j[-1][0].cell.array)
    print(compute_distortion(i, j[-1][0]))
    print()
    print(k[-1][0].cell.array)
    print(compute_distortion(i, k[-1][0]))
    
    print()
    if False:
        print('ijs')
        for ij in j:
            print('ij')
            print(ij[0].cell.array)
            print(ij[1])
            print(ij[2])
            print(ij[3])
            print('@')
            print()
    print()
    print('#'*50)

['Cu']
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]

[[1.80982878 1.80982878 0.        ]
 [0.         1.80982878 1.80982878]
 [1.80982878 0.         1.80982878]]
1.7281680248448638e-16

[[ 1.81194863e+00  1.80601090e+00  5.24523000e-04]
 [ 4.67841200e-03  1.80780547e+00  1.81316908e+00]
 [ 1.81021323e+00 -4.56658900e-03  1.81005496e+00]]
0.0011047402690813656


##################################################


In [34]:
for i in df.loc[:7, 'init_structure']:
    print(i.cell.array)

[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
[[1.5 1.5 0. ]
 [0.  1.5 1.5]
 [1.5 0.  1.5]]
